Montar drive

In [ ]:
#from google.colab import drive 
#drive.mount('/content/drive/')

Declarar los flags una única vez 


> (se comenta esta parte en todos los .py)



In [ ]:
from absl import app, flags, logging
from absl.flags import FLAGS
flags.DEFINE_string('classes', './Proyecto/data/mask.names', 'path to classes file')
flags.DEFINE_string('weights', './checkpoints/yolov3.tf',
                    'path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('image', './data/girl.png', 'path to input image')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_string('output', './output.jpg', 'path to output image')
flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')
flags.DEFINE_string('output_format', 'DIVX', 'codec used in VideoWriter when saving video to file')
flags.DEFINE_string('video', './data/video.mp4',
                    'path to video file or number for webcam)')
flags.DEFINE_string('dataset', '', 'path to dataset')
flags.DEFINE_string('val_dataset', '', 'path to validation dataset')


flags.DEFINE_enum('mode', 'fit', ['fit', 'eager_fit', 'eager_tf'],
                  'fit: model.fit, '
                  'eager_fit: model.fit(run_eagerly=True), '
                  'eager_tf: custom GradientTape')
flags.DEFINE_enum('transfer', 'none',
                  ['none', 'darknet', 'no_output', 'frozen', 'fine_tune'],
                  'none: Training from scratch, '
                  'darknet: Transfer darknet, '
                  'no_output: Transfer all but output, '
                  'frozen: Transfer and freeze all, '
                  'fine_tune: Transfer all and freeze darknet only')
flags.DEFINE_integer('epochs', 2, 'number of epochs')
flags.DEFINE_integer('batch_size', 8, 'batch size')
flags.DEFINE_float('learning_rate', 1e-3, 'learning rate')
flags.DEFINE_integer('weights_num_classes', None, 'specify num class for `weights` file if different, '
                     'useful in transfer learning with different number of classes')
flags.DEFINE_enum('split', 'train', [
                  'train', 'val'], 'specify train or val spit')
flags.DEFINE_string('data_dir', './Proyecto/',
                    'path to raw PASCAL VOC dataset')
flags.DEFINE_string('output_file', './Proyecto/tfrecord/train.tfrecord', 'outpot dataset')
flags.DEFINE_string('folder', './Proyecto/test/', 'folder')
flags.DEFINE_string('output_folder', './Proyecto/output/', 'output folder')


Instalar requirements

In [ ]:
%cd drive/MyDrive/ProyectoFinal/yolov3-tf2-master/
!pip install -r requirements-gpu.txt

Verificar versión de tensorflow

In [ ]:
import tensorflow as tf
tf.__version__

Obtener pesos

In [ ]:
#!wget https://pjreddie.com/media/files/yolov3.weights -O data/yolov3.weights
!python convert.py

In [ ]:
#Navegar a la carpeta del proyecto
#%cd drive/MyDrive/ProyectoFinal/

In [ ]:
#%ls

# Crear Tfrecord para train y test

In [ ]:
%run yolov3-tf2-master/tools/voc2012.py --data_dir ./Proyecto --split train --output_file=./Proyecto/tfrecord/train.tfrecord --classes=./Proyecto/data/mask.names

In [ ]:
%run yolov3-tf2-master/tools/voc2012.py --data_dir ./Proyecto --split val --output_file=./Proyecto/tfrecord/val.tfrecord --classes=./Proyecto/data/mask.names

# Entrenamiento

In [ ]:
%run yolov3-tf2-master/train.py \
	--dataset ./Proyecto/tfrecord/train.tfrecord \
	--val_dataset ./Proyecto/tfrecord/val.tfrecord \
	--classes ./Proyecto/data/mask.names \
	--num_classes 2 \
	--mode fit --transfer darknet \
	--batch_size 16 \
	--epochs 85 \
  --weights_num_classes=80 

# Validación

In [ ]:
%run yolov3-tf2-master/detect.py \
	--classes ./Proyecto/data/mask.names \
	--num_classes 2 \
	--weights ./checkpoints/yolov3_train_16.tf \
	--tfrecord ./Proyecto/tfrecord/val.tfrecord

In [ ]:
%run yolov3-tf2-master/detect.py  \
  --classes ./Proyecto/data/mask.names \
  --image ./test_images/test5.jpg \
  --output ./test_images/test5-v.jpg \
  --weights ./checkpoints/yolov3_train_16.tf \
  --num_classes 2

In [ ]:
%run yolov3-tf2-master/detect_video.py  \
  --classes ./Proyecto/data/mask.names \
  #--video 0 \
  --video ./valvideo3.mp4 \
  --output ./Proyecto/videorec3.avi \
  --weights ./checkpoints/yolov3_train_16.tf \
  --num_classes 2

In [ ]:
%run yolov3-tf2-master/detect_folder.py  \
  --classes ./Proyecto/data/mask.names \
  --weights ./checkpoints/yolov3_train_85.tf \
  --folder /Users/macbookpro/ml-env/ProyectoFinal/test/ \
  --output_folder /Users/macbookpro/ml-env/ProyectoFinal/output/ \
  --num_classes 2

In [ ]:
%run yolov3-tf2-master/detect.py  \
  --classes ./Proyecto/data/mask.names \
  --image ./Proyecto/JPEGImages/nomask/66000_Mask.jpg \
  --output ./Proyecto/output_66000_Mask.jpg \
  --weights ./checkpoints/yolov3_train_19.tf \
  --num_classes 2